# Densnet for ISIC Skin Cancer Dataset
## Imports:

In [1]:
import tensorflow as tf
import time
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
import numpy as np                                    
import pandas as pd 
import os
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
warnings.filterwarnings("ignore")


## Densnet Model (own)

In [2]:

def densenet(input_shape, n_classes, filters = 32):
  # Same sequence for each convolutional block after the input
  # Batch normalization, ReLu activation + Conv2D layer
  def bn_rl_conv(x, filters, kernel=1, strides=1):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, kernel, strides=strides, padding = "same")(x)
    return x
  
  def dense_block(x, repetition):
    # Each dense block has 2 convolutions with 1x1 and 3x3 kernels
    # 1st dense block with 6 repetitions
    # 2nd dense block with 12 repetiotions
    # 3rd dense block with 24 repetitions
    # 4th dense block with 16 repetitions

    # Each block is run for the 6,12,24,16
    for _ in range(repetition):
      y = bn_rl_conv(x, 4*filters)    # Every 1x1 convolutions has 4-times the number of filters
      y = bn_rl_conv(y, filters, 3)   # But 3x3 filters are oly present once

    return x
  
  # Transition layer
  # REmove channels to half of the existing channels 
  def transition_layer(x):
    # K.int_shape(x) returns a tuple with the dimensions of x
    # [-1] is the last dimension which is the filters.
    # Since we need half of them we devide it by 2
    x = bn_rl_conv(x, K.int_shape(x)[-1]//2)        # 1x1 convolution layer
    x = AvgPool2D(2, strides=2, padding='same')(x)  # 2x2 average poolling layer with strid of 2
    return x

  


  input = Input(input_shape)
  # 1st convolution block with 64 filters of size 7x7 & a stride of 2:
  x = Conv2D(64, 7, strides = 2, padding = "same")(input)
  # Max pooling laxer with 3x3 max pooling & stride of 2
  x = MaxPool2D(3, strides=2, padding="same")(x)

  # Run 4-times trough the 6, 12, 24, 16 repetitions
  for repetition in [6,12,24,16]:
    d = dense_block(x, repetition)
    x = transition_layer(d)
  print(x)
  # Glabal average pooling
  x = GlobalAveragePooling2D()(d)
  print(x)
  # Final dense output layer
  output = Dense(n_classes, activation="softmax")(x)
  print(x)

  model = Model(input, output)
  return model


## Compare to DenseNet121 from keras

## Load the ground truth dataset
* Import data and set column header
* Sort by image_name (Should then be consistent with the images which will be sorted the same way.)

In [3]:
# Set column headers
head_list = ["image_name","patient_id","sex", "age_approx","anatom_site_general_challenge", "diagnosis", "benign_malignant", "target"]

# Load dataset @todo make reusable
meta_train = pd.read_csv("C:\\Users\\clara\\DataSets\\ISIC_2020_Training_GroundTruth.csv", names=head_list)

# Sort values by image name
meta_train.sort_values(by=['image_name'], inplace=True)

# Display the dataset.
meta_train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
2,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
3,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
4,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
5,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
6,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


## Preprocess the Ground truth

* Drop "unknown" diagnosis from CSV dataset
* Store the image names to drop from the image dataset

In [4]:
# Remove undefined diagnosis from dataset
drop_images = []
drop_idx = []
for index, row in meta_train.iterrows(): 
    if row["diagnosis"] == "unknown":
        drop_images.append(row["image_name"])
        drop_idx.append(index)
# Dop unknown values
meta_train.drop(drop_idx)

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
3,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
13,ISIC_0076995,IP_2235340,female,55.0,torso,nevus,benign,0
27,ISIC_0084086,IP_4023055,male,60.0,lower extremity,nevus,benign,0
28,ISIC_0084270,IP_2961528,male,40.0,lower extremity,nevus,benign,0
29,ISIC_0084395,IP_0175539,female,45.0,torso,nevus,benign,0
...,...,...,...,...,...,...,...,...
33114,ISIC_9997614,IP_1705144,female,50.0,upper extremity,nevus,benign,0
33118,ISIC_9998682,IP_2516168,male,60.0,head/neck,melanoma,malignant,1
33119,ISIC_9998937,IP_3091321,male,40.0,head/neck,nevus,benign,0
33126,ISIC_9999806,IP_0046310,male,45.0,torso,nevus,benign,0


## Load the image dataset
* Remove the images for which the diagnosis is unknown


In [5]:
#@TODO replace this with something reusable e.g. os.getcwd()
imgPathFull = "C:\\Users\\clara\\DataSets\\ISIC_2020_Training_JPEG\\train\\"

# Load and sort images
imagePaths = sorted(list(os.listdir(imgPathFull)))

# Remove unknown diagnosis
filtered_paths = [i for i in imagePaths if i not in drop_images]       

In [6]:
# Check if the shapes of ground truth and images are correct
if (meta_train.shape[0] != len(filtered_paths)):
    print("Ground truth and images paths do not have compatible sizes")
    #@TODO end process otherwise

Ground truth and images paths do not have compatible sizes


## Load & preprocess the images
* Resize them to (224, 224)
* Convert them to numpy arrays
* Generate the labels

In [9]:
#@TODO optimize this (takes too much time) 
# We could e.g. use tfrecords or sth.
data=[]
labels=[]
counter = 0
maxcount = len(filtered_paths)
# Comment this out to use the whole dataset
maxcount = 2000
tic = time.time()
# Decide how often you want to print the progress
print_per = 100
for index, row in meta_train.iterrows():
    if (counter >= maxcount):
        break    
    img = imagePaths[counter]
    img_path = imgPathFull+ img   
    image = cv2.imread(img_path)
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    data.append(image)
    labels.append(row["diagnosis"])
    counter += 1
    if (counter%print_per == 0):
        duration = time.time()-tic
        print("{}/{} files processed; {:.4} data/s".format(counter, maxcount, float(counter)/duration))


100/2000 files processed; 6.077 data/s
200/2000 files processed; 7.198 data/s
300/2000 files processed; 7.075 data/s
400/2000 files processed; 7.151 data/s
500/2000 files processed; 7.404 data/s
600/2000 files processed; 7.498 data/s
700/2000 files processed; 7.334 data/s
800/2000 files processed; 7.261 data/s
900/2000 files processed; 7.166 data/s
1000/2000 files processed; 7.12 data/s
1100/2000 files processed; 7.157 data/s
1200/2000 files processed; 7.147 data/s
1300/2000 files processed; 7.071 data/s
1400/2000 files processed; 7.076 data/s
1500/2000 files processed; 7.088 data/s
1600/2000 files processed; 7.027 data/s
1700/2000 files processed; 6.971 data/s
1800/2000 files processed; 6.921 data/s
1900/2000 files processed; 6.89 data/s
2000/2000 files processed; 6.891 data/s


### Conversion of Labels & Data
* Binarize labels in a one-vs-all fashion.
* Convert data to np.array

In [10]:
#@TODO we need to change this as e.g. my computer cannot handle the conversion to np.array for the whole dataset

# Convert from cathegorical to numerical
labels = np.array(labels)

# Count the unique values in the diagnosis column
num_lables = len(np.unique(labels))

mlb = LabelBinarizer()
bin_labels = mlb.fit_transform(labels)
data = np.array(data, dtype="float32") / 255.0

## Generate Model
* Use the number of labels for generation
* Display a summary

In [11]:
# We currently need to wait till here since we do not use the whole dataset & therfore we do not know how many labels are currently present.
input_shape = 224,224, 3

model = densenet(input_shape, num_lables)
model.summary()

Tensor("average_pooling2d_3/Identity:0", shape=(None, 4, 4, 4), dtype=float32)
Tensor("global_average_pooling2d/Identity:0", shape=(None, 8), dtype=float32)
Tensor("global_average_pooling2d/Identity:0", shape=(None, 8), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 56, 56, 64)        256       
_________________________________________________________________
re_lu_12 (ReLU)              (None, 56, 56, 64)        0         
________________________

### Avoid overfitting
* change once network has been optimized

In [12]:
for layer in model.layers[:-8]:
    layer.trainable=False
    
for layer in model.layers[-8:]:
    layer.trainable=True
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 56, 56, 64)        256       
_________________________________________________________________
re_lu_12 (ReLU)              (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 56, 56, 32)        2080      
_________________________________________________________________
average_pooling2d (AveragePo (None, 28, 28, 32)        0     

## Create train/test split

In [13]:
#@TODO change this to use the test dataset instead of this split
(xtrain,xtest,ytrain,ytest)=train_test_split(data,bin_labels,test_size=0.2,random_state=42)

## Additional Preprocessing
* Reduce learning rate when a metric has stopped improving.

* Create Checkpoint
* Create image batches with real-time augmentation

In [14]:

anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)

checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)

## Fit the Data & Compile tha Model

In [15]:
datagen.fit(data)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
#@TODO use model.fit instead (due to deprication)
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=224),
               steps_per_epoch=xtrain.shape[0] //224,
               epochs=50,
               verbose=num_lables,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50

Epoch 00001: val_loss improved from inf to 1.57376, saving model to model.h5
Epoch 2/50

Epoch 00002: val_loss improved from 1.57376 to 1.49593, saving model to model.h5
Epoch 3/50

Epoch 00003: val_loss improved from 1.49593 to 1.41927, saving model to model.h5
Epoch 4/50

Epoch 00004: val_loss improved from 1.41927 to 1.34532, saving model to model.h5
Epoch 5/50

Epoch 00005: val_loss improved from 1.34532 to 1.27295, saving model to model.h5
Epoch 6/50

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.001.

Epoch 00006: val_loss improved from 1.27295 to 1.20209, saving model to model.h5
Epoch 7/50

Epoch 00007: val_loss improved from 1.20209 to 1.13226, saving model to model.h5
Epoch 8/50

Epoch 00008: val_loss improved from 1.13226 to 1.06750, saving model to model.h5
Epoch 9/50

Epoch 00009: val_loss improved from 1.06750 to 1.00871, saving model to model.h5
Epoch 10/50

Epoch 00010: 

## Test the Data

* Predict the test split
* Display how many we predicted correctly.


In [17]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

Total-test-data; 400 	accurately-predicted-data: 353 	 wrongly-predicted-data:  47
Accuracy: 88.25 %


## Make a confusion matrix

#### General TODOs
* use train-test dataset
* upload dataset to azure
* CLUSTERING! before (consider that there are hairy images aswell)


#### References:
* [pluralsight](https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow)
* [towardsdatascience](https://towardsdatascience.com/creating-densenet-121-with-tensorflow-edbc08a956d8)